In [1]:
import pandas as pd

data = pd.read_csv('C:/Users/mchampain/Documents/GitHub/de-fr.txt/OpenSubtitles.de-fr.txt', header = None, sep='\t')

Setup - Import TensorFlow and other libraries

In [2]:
conda install tensorflow

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [5]:
# Read, then decode for py2 compat.
text = open('C:/Users/mchampain/Documents/GitHub/de-fr.txt/OpenSubtitles.de-fr.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
#print ('Length of text: {} characters'.format(len(text)))

In [6]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 558646528 characters


In [7]:
print(text[:250])

Bonjour. Je suis rédacteur au journal L'Illustration.
Je cherche M. Odilon Hubert, député d'Alger.
Il est là-bas.
Merci.
Monsieur Hubert ?
Camille Bruyère.
Vous êtes le 1er représentant des Français d'Algérie au Parlement.
Mon journal voudrait vous c


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


434 unique characters


Process the text
Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
text_as_int

array([32, 77, 76, ...,  1, 26,  0])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'Bonjour. Je s' ---- characters mapped to int ---- > [32 77 76 72 77 83 80 14  1 40 67  1 81]


The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?
Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])


B
o
n
j
o


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))


"Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"
" est là-bas.\nMerci.\nMonsieur Hubert ?\nCamille Bruyère.\nVous êtes le 1er représentant des Français d'A"
"lgérie au Parlement.\nMon journal voudrait vous consacrer un article.\nSur l'Algérie ?\nNon. Sur vous.\nS"
"ur moi ?\nC'est une histoire de violence et de sang, à la Shakespeare.\nLes lecteurs adorent ça, monsie"
"ur.\nVous croyez ?\nIl s'agit d'un autre pays, jeune homme, d'un autre monde, d'une autre terre où tout"


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [16]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>

Print the first examples input and target values:

In [17]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  "Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nI"
Target data: "onjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [18]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 32 ('B')
  expected output: 77 ('o')
Step    1
  input: 77 ('o')
  expected output: 76 ('n')
Step    2
  input: 76 ('n')
  expected output: 72 ('j')
Step    3
  input: 72 ('j')
  expected output: 77 ('o')
Step    4
  input: 77 ('o')
  expected output: 83 ('u')


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

    tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
    tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
    tf.keras.layers.Dense: The output layer, with vocab_size outputs.


In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [22]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

A drawing of the data passing through the model
Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 434) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [24]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           111104    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 434)           444850    
Total params: 4,494,258
Trainable params: 4,494,258
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.
Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


This gives us, at each timestep, a prediction of the next character index:

In [26]:
sampled_indices

array([342, 129, 202, 254, 262, 328, 268, 269,  66,  40, 373,  23, 334,
       274, 143,  34, 418, 398, 339,  88, 160, 380,  29, 130, 394,  57,
       333,  99, 271, 391, 157, 249,   0,  85,  97,  18, 284,   8, 253,
       115, 334, 266, 266, 209,  28, 318, 343, 101, 378, 116, 241, 238,
       274, 240,  40,  34, 252, 315,  29, 291, 333, 167,  62, 322, 399,
       139, 155, 264,  85,  91, 383,  48, 204, 188, 254, 292, 383, 394,
       173, 135, 319, 136, 121, 381, 236, 282, 143, 121,  50, 426, 262,
        70,  44,  60, 297, 184, 137, 118, 337, 122], dtype=int64)

Decode these to see the text predicted by this untrained model:

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 " Assieds-toi, Rico.\n- Ouais, merci.\nA présent, écoute.\nJ'ai à te parler mais tu n'entendras pas un m"

Next Char Predictions: 
 '\u200f¥ðŽˆوΔΕdJ\u2e777ഹΜ´D獶挠ῖzÆ〵?¦㸭[ര\x87Η㱴Ãů\nw\x852η(Ż\x97ഹΑΑ÷=ѕ–\x89〬\x98ŕŒΜŔJDźх?τരÍ`ҫ敬°Á˜w~ㄽRòâŽωㄽ㸭Ó¬і\xad\x9d〺ńβ´\x9dT眾ˆhN^ЗÞ®\x9aᴥ\x9e'


Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.
Attach an optimizer, and a loss function

The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 434)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       6.0728974


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [29]:
model.compile(optimizer='adam', loss=loss)


Configure checkpoints
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [32]:
EPOCHS=1

In [34]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [35]:
history = model.fit(dataset, epochs=EPOCHS)

     79/Unknown - 484s 6s/step - loss: 2.2801

KeyboardInterrupt: 

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

ValueError: The `batch_size` argument must not be specified for the given input type. Received input: <BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>, batch_size: 1000

Generate text
Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_1'

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            111104    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 434)            444850    
Total params: 4,494,258
Trainable params: 4,494,258
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

    It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

    Get the prediction distribution of the next character using the start string and the RNN state.

    Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

    The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.

To generate text the model's output is fed back to the input

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [41]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 100

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [47]:
print(generate_text(model, start_string=u"Bonjour salope"))


Bonjour salope.
Cela suffit à des recters ?
C'est tout a ta dit en son comme un métier !
Hé, non !
Je ne suis pas 


The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop output.

We will use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

The procedure works as follows:

    First, initialize the RNN state. We do this by calling the tf.keras.Model.reset_states method.

    Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

    Open a tf.GradientTape, and calculate the predictions and loss in that context.

    Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method.

    Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method.


In [48]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [49]:
optimizer = tf.keras.optimizers.Adam()

In [50]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [51]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 6.071437358856201


KeyboardInterrupt: 